In [0]:
dbuitls.widgets.text("param_data_source", "")
var_data_source = dbutils.widgets.get("param_data_source")

In [0]:
dbutils.widgets.text("param_file_date", "2021-03-21") # based on the name of the subfolder in blob storage
var_file_date = dbutils.widgets.get("param_file_date")

### Read the JSON file 

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/utils"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [0]:
result_schema = StructType([StructField("resultId", IntegerType(), False), 
                              StructField("raceId", IntegerType(), True),
                              StructField("driverId", IntegerType(), True),
                              StructField("constructorId", IntegerType(), True),
                              StructField("number", IntegerType(), True),
                              StructField("grid", IntegerType(), True),
                              StructField("position", IntegerType(), True),
                              StructField("positionText", StringType(), True),
                              StructField("points", FloatType(), True),
                              StructField("laps", IntegerType(), True),
                              StructField("time", StringType(), True),
                              StructField("milliseconds", IntegerType(), True),
                              StructField("fastestLap", IntegerType(), True),
                              StructField("rank", IntegerType(), True),
                              StructField("fastestLapTime", StringType(), True),
                              StructField("fastestLapSpeed", FloatType(), True),
                              StructField("statusId", IntegerType(), True),
                            ])


results_df = spark.read \
    .schema(result_schema) \
    .json(f"{RAW_FOLDER_PATH}/{var_file_date}/results.json")   


In [0]:
results_df.printSchema()

In [0]:
display(results_df)

### rename columns and add new column

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, date_trunc, from_utc_timestamp

In [0]:
modified_results_df = add_ingestion_date(results_df.withColumnRenamed("resultId", "result_id") \
                                .withColumnRenamed("raceId", "race_id") \
                                .withColumnRenamed("driverId", "driver_id") \
                                .withColumnRenamed("constructorId", "constructor_id") \
                                .withColumnRenamed("positionText", "position_text") \
                                .withColumnRenamed("positionOrder", "position_order") \
                                .withColumnRenamed("fastestLap", "fastest_lap") \
                                .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed")) \
                                .withColumn("data_source", lit(var_data_source)) \
                                .withColumn("file_date", lit(var_file_date)) 



display(modified_results_df)

### drop the unwanted column

In [0]:
final_results_df = modified_results_df.drop(col("statusId"))

In [0]:
display(final_results_df)

### write to parquet file

In [0]:
for race_id_list in final_results_df.select("race_id").distinct().collect(): # drop the partition before it loads the data
    if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
        spark.sql("ALTER TABLE f1_processed.results 
                DROP IF EXISTS PARTITION (race_id = {race_id_list.race_id})")

In [0]:
final_results_df.write.mode("append").partitionBy("race_id").format("parquet").saveAsTable("f1_processed.results")


In [0]:
# spark.read.parquet(f"{PROCESSED_FOLDER_PATH}/results").display()

In [0]:
%sql 
SELECT * FROM f1_processed.results

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql 
SELECT race_id, COUNT(1)
FROM f1_processed.results
GROUP BY race_id; 
